# Alimentation de la base de données

In [16]:
import pandas as pd
from pathlib import Path
import numpy as np
import sys

sys.path.append("../")
from utils import read_data
from db import Engine, Transaction, Transmission, Customer, Model, MySQLConnector

In [20]:
conn = MySQLConnector()
conn.connect()
conn

MySQLConnector(host='localhost', user='admin', password='secret', database='cars24', db=<mysql.connector.connection_cext.CMySQLConnection object at 0x718e66009ff0>)

### Analyse des données existantes ?

In [21]:
conn.execute("SELECT * FROM engine LIMIT 1")

,id_engine,fuel_type,capacity


In [22]:
conn.execute("SELECT * FROM transmission LIMIT 1")

,id_transmission,transmission_type


In [23]:
conn.execute("SELECT * FROM model LIMIT 1")

,id_model,model_name,id_engine,id_transmission,manufacturing_year


In [24]:
conn.execute("SELECT * FROM customer LIMIT 1")

,id_customer,first_name,last_name,email,rating


In [25]:
conn.execute("SELECT * FROM transaction LIMIT 1")

,id_transaction,id_customer,id_model,price,km_driven,spare_key,ownership,imperfections,repainted_parts


### Ajouts des éléments

A ne lancer qu'une fois si la base de données est vide 

In [26]:
# Main Path
db_path = Path("../data/cars24data.csv")
df = read_data(db_path)
df.head()

,Model Name,Price,Manufacturing_year,Engine capacity,Spare key,Transmission,KM driven,Ownership,Fuel type,Imperfections,Repainted Parts
0,2017 Maruti Swift VXI,561000,2017,1197,No,Manual,25847,2,Petrol,6,2
1,2016 Maruti Baleno DELTA PETROL 1.2,498000,2016,1197,Yes,Manual,55511,2,Petrol,12,1
2,2020 Maruti Swift VXI,577000,2020,1197,No,Manual,47110,1,Petrol,4,2
3,2022 Maruti Ertiga VXI AT SHVS,1084000,2022,1462,Yes,Automatic,35378,1,Petrol,2,3
4,2019 Maruti Dzire VXI,603000,2019,1197,Yes,Manual,91856,1,Petrol,3,2


In [27]:
# Moteur
# ======
df_motor = df[["Engine capacity", "Fuel type"]].drop_duplicates()
df_motor = df_motor.rename(
    columns={
        "Engine capacity": "capacity",
        "Fuel type": "fuel_type",
    }
)
print(df_motor.shape)

list_ids = []
for row in df_motor.iterrows():
    capacity = row[1]["capacity"]
    fuel_type = row[1]["fuel_type"]
    engine = Engine(fuel_type=fuel_type, capacity=capacity)
    id_engine = conn.add_entry(engine)
    list_ids.append(id_engine)
df_motor["id_engine"] = list_ids
df_motor.head()

(14, 2)


,capacity,fuel_type,id_engine
0,1197,Petrol,2343f91d-5291-47ee-96a4-33b827968fca
3,1462,Petrol,3a4c6a69-05c2-4597-95d6-b0f71a474f76
5,796,Petrol,a254380f-ea84-4018-8fed-4551854ac95f
9,998,Petrol,433f8970-ea23-4e07-8294-e82821e11f1d
34,1248,Diesel,5b16ac17-1268-4078-afd1-2aba6819a246


In [28]:
# Transmission
# ============
df_transmission = df[["Transmission"]].drop_duplicates()
df_transmission = df_transmission.rename(
    columns={
        "Transmission": "transmission_type",
    }
)
print(df_transmission.shape)
list_ids = []
for row in df_transmission.iterrows():
    tr = row[1]["transmission_type"]
    trans = Transmission(transmission_type=tr)
    id_trans = conn.add_entry(trans)
    list_ids.append(id_trans)
df_transmission["id_transmission"] = list_ids
df_transmission.head()

(2, 1)


,transmission_type,id_transmission
0,Manual,81df1b3c-ec67-46bc-9afd-0ef81974be74
3,Automatic,aa54da78-124c-4a4e-a240-9b1026a8b453


In [29]:
# Modeles
# =======
df_frame = df[
    ["Model Name", "Manufacturing_year", "Transmission", "Engine capacity", "Fuel type"]
].drop_duplicates()
df_frame = df_frame.rename(
    columns={
        "Model Name": "model_name",
        "Manufacturing_year": "manufacturing_year",
        "Transmission": "transmission_type",
        "Engine capacity": "capacity",
        "Fuel type": "fuel_type",
    }
)
df_frame = pd.merge(
    df_frame, df_motor, on=["capacity", "fuel_type"], how="left", validate="m:1"
)
df_frame = pd.merge(
    df_frame, df_transmission, on=["transmission_type"], how="left", validate="m:1"
)
df_frame = df_frame.drop(["capacity", "fuel_type", "transmission_type"], axis=1)
print(df_frame.shape)
list_ids = []
for row in df_frame.iterrows():
    name = row[1]["model_name"]
    year = row[1]["manufacturing_year"]
    engine_id = row[1]["id_engine"]
    transmission_id = row[1]["id_transmission"]

    mod = Model(
        id_engine=engine_id,
        id_transmission=transmission_id,
        model_name=name,
        manufacturing_year=year,
    )
    id_mod = conn.add_entry(mod)
    list_ids.append(id_mod)
df_frame["id_model"] = list_ids
df_frame.head()

(584, 4)


,model_name,manufacturing_year,id_engine,id_transmission,id_model
0,2017 Maruti Swift VXI,2017,2343f91d-5291-47ee-96a4-33b827968fca,81df1b3c-ec67-46bc-9afd-0ef81974be74,a347af7d-a3e2-44ff-9423-5387be1b9d1d
1,2016 Maruti Baleno DELTA PETROL 1.2,2016,2343f91d-5291-47ee-96a4-33b827968fca,81df1b3c-ec67-46bc-9afd-0ef81974be74,1c16a4b8-5409-40b9-8da1-c8ec210ea902
2,2020 Maruti Swift VXI,2020,2343f91d-5291-47ee-96a4-33b827968fca,81df1b3c-ec67-46bc-9afd-0ef81974be74,59650bad-855b-4ed5-a4ac-930aa490bc42
3,2022 Maruti Ertiga VXI AT SHVS,2022,3a4c6a69-05c2-4597-95d6-b0f71a474f76,aa54da78-124c-4a4e-a240-9b1026a8b453,431597df-5774-4bc7-9637-cf9018295845
4,2019 Maruti Dzire VXI,2019,2343f91d-5291-47ee-96a4-33b827968fca,81df1b3c-ec67-46bc-9afd-0ef81974be74,d84c220f-4512-4a7c-9a4c-cb8d251b46d7


In [30]:
# Clients
# =======
df_customer = read_data(Path("../data/customers.csv"))
df_customer.head()

,first_name,last_name,email,rating
0,Joseph,Savage,richardlevy@example.net,4
1,Bobby,Donaldson,krauseamy@example.org,2
2,Angelica,Jarvis,esandoval@example.net,2
3,James,Boyd,qroberts@example.org,4
4,Nancy,Paul,rodriguezbruce@example.net,3


In [31]:
list_ids = []
for row in df_customer.iterrows():
    first_name = row[1]["first_name"]
    last_name = row[1]["last_name"]
    email = row[1]["email"]
    rating = str(row[1]["rating"]) + " star"

    cust = Customer(
        first_name=first_name, last_name=last_name, email=email, rating=rating
    )
    id_cust = conn.add_entry(cust)
    list_ids.append(id_cust)
df_customer["id_customer"] = list_ids
df_customer.head()

,first_name,last_name,email,rating,id_customer
0,Joseph,Savage,richardlevy@example.net,4,8d65ff84-99fc-402e-bf22-37ae8bfd5e43
1,Bobby,Donaldson,krauseamy@example.org,2,89be7c40-19b9-4117-b5e6-855fc80ffbe4
2,Angelica,Jarvis,esandoval@example.net,2,900f20b0-da96-4175-93d8-2b8364b47c42
3,James,Boyd,qroberts@example.org,4,ad384e62-1875-43d3-9878-6da45dc20e4b
4,Nancy,Paul,rodriguezbruce@example.net,3,239a6410-a8af-49fd-a1a3-7a2e3ff891bb


In [32]:
# Transactions
# ============
df_vehicule = df.rename(
    columns={
        "Model Name": "model_name",
        "Manufacturing_year": "manufacturing_year",
        "Transmission": "transmission_type",
        "Spare key": "spare_key",
        "KM driven": "km_driven",
        "Ownership": "ownership",
        "Imperfections": "imperfections",
        "Repainted Parts": "repainted_parts",
        "Engine capacity": "capacity",
        "Fuel type": "fuel_type",
        "Price": "price",
    }
)
df_vehicule = pd.merge(
    df_vehicule,
    df_motor,
    on=["capacity", "fuel_type"],
    how="left",
    validate="m:1",
)
df_vehicule = pd.merge(
    df_vehicule,
    df_transmission,
    on=["transmission_type"],
    how="left",
    validate="m:1",
)
df_vehicule = pd.merge(
    df_vehicule,
    df_frame,
    on=["id_engine", "id_transmission", "model_name", "manufacturing_year"],
    how="left",
    validate="m:1",
)
df_vehicule = df_vehicule.drop(
    [
        "transmission_type",
        "manufacturing_year",
        "model_name",
        "capacity",
        "fuel_type",
        "id_engine",
        "id_transmission",
    ],
    axis=1,
)
list_customers = list(df_customer["id_customer"].unique())
df_vehicule["id_customer"] = 0
df_vehicule["id_customer"] = df_vehicule["id_customer"].apply(
    lambda x: np.random.choice(list_customers)
)
print(df_vehicule.shape)

(1445, 8)


In [33]:
for row in df_vehicule.iterrows():
    id_model = row[1]["id_model"]
    id_customer = row[1]["id_customer"]
    price = row[1]["price"]
    km_driven = row[1]["km_driven"]
    spare_key = row[1]["spare_key"] == "Yes"
    ownership = row[1]["ownership"]
    imperfections = row[1]["imperfections"]
    repainted_parts = row[1]["repainted_parts"]

    trans = Transaction(
        id_model=id_model,
        id_customer=id_customer,
        price=price,
        km_driven=km_driven,
        spare_key=spare_key,
        ownership=ownership,
        imperfections=imperfections,
        repainted_parts=repainted_parts,
    )
    _ = conn.add_entry(trans)